# Architecture Reconstruction for Zeeguu

This notebook provides a comprehensive architecture reconstruction for the Zeeguu system, analyzing both the backend API (`../Data/api`) and frontend (`../Data/frontend`). The goal is to extract meaningful architectural views that provide insights into the system's structure, dependencies, and complexity.

## 1. Setup and Installation

Install all the necessary packages for our analysis.

In [ ]:
import sys
!{sys.executable} -m pip install prospector pylint pydepend networkx matplotlib pyyaml jinja2 pandas pyreverse graphviz pydot

## 2. Configuration and Setup

Define paths and configuration options for the analysis. Note that, since this notebook is in `Root/Tools`, the project paths are defined relative to that location.

In [ ]:
import os
import sys
import json
import subprocess
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, HTML, Image
from pathlib import Path

# Define paths relative to the notebook location (Root/Tools)
API_PATH = os.path.join('..', 'Data', 'api')
FRONTEND_PATH = os.path.join('..', 'Data', 'frontend')
OUTPUT_DIR = 'architecture_analysis'

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Backend path set to: {API_PATH}")
print(f"Frontend path set to: {FRONTEND_PATH}")

## 3. Basic System Information

Extract basic information about the system to understand its size and scope.

In [ ]:
def count_files_and_lines(path, extensions=None):
    """
    Count files and lines of code in the given path, filtered by file extensions.
    """
    if extensions is None:
        extensions = ['.py', '.js', '.jsx', '.ts', '.tsx']
    
    file_count = 0
    line_count = 0
    files_by_ext = {ext: 0 for ext in extensions}
    lines_by_ext = {ext: 0 for ext in extensions}
    
    for root, _, files in os.walk(path):
        for file in files:
            _, ext = os.path.splitext(file)
            if ext in extensions:
                file_count += 1
                files_by_ext[ext] += 1
                
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        lines = len(f.readlines())
                        line_count += lines
                        lines_by_ext[ext] += lines
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")
    
    return {
        'total_files': file_count,
        'total_lines': line_count,
        'files_by_ext': files_by_ext,
        'lines_by_ext': lines_by_ext
    }

# Analyze backend
backend_stats = count_files_and_lines(API_PATH)
print("Backend Statistics:")
print(json.dumps(backend_stats, indent=2))

# Analyze frontend
frontend_stats = count_files_and_lines(FRONTEND_PATH)
print("\nFrontend Statistics:")
print(json.dumps(frontend_stats, indent=2))

## 4. Code Quality Analysis with Prospector

Run a comprehensive code quality analysis using Prospector.

In [ ]:
def run_prospector(path, output_file):
    """
    Run Prospector on the specified path and save results to output_file.
    Returns a summary of findings.
    """
    full_output_path = os.path.join(OUTPUT_DIR, output_file)
    
    # Run prospector and capture its JSON output
    try:
        result = subprocess.run(
            ['prospector', path, '--output-format=json'], 
            capture_output=True, text=True, check=True
        )
        
        # Write the output to a file
        with open(full_output_path, 'w', encoding='utf-8') as f:
            f.write(result.stdout)
        
        print(f"Prospector analysis completed for {path}")
        
        # Load and parse the results
        results = json.loads(result.stdout)
        messages = results.get('messages', [])
        tools_used = results.get('tools', [])
        summary = {
            'total_issues': len(messages),
            'tools_used': tools_used,
            'issues_by_type': {}
        }
        
        for msg in messages:
            msg_type = msg.get('source')
            if msg_type not in summary['issues_by_type']:
                summary['issues_by_type'][msg_type] = 0
            summary['issues_by_type'][msg_type] += 1
            
        return summary
    except Exception as e:
        print(f"Error running Prospector on {path}: {e}")
        return None

# Run Prospector on backend
backend_prospector = run_prospector(API_PATH, 'api_prospector_report.json')
print("\nBackend Prospector Summary:")
print(json.dumps(backend_prospector, indent=2))

# Run Prospector on frontend
frontend_prospector = run_prospector(FRONTEND_PATH, 'frontend_prospector_report.json')
print("\nFrontend Prospector Summary:")
print(json.dumps(frontend_prospector, indent=2))

## 5. Module Dependency Analysis

Extract and visualize module dependencies using PyDepend for the backend and a custom import parser for the frontend.

In [ ]:
def extract_dependencies(path, is_backend=True):
    """
    Extract module dependencies from the given path.
    For the backend (Python), use pydepend.
    For the frontend (JavaScript/React), parse import statements.
    """
    if is_backend:
        output_path = os.path.join(OUTPUT_DIR, 'api_dependencies.dot')
        cmd = ['pydepend', '--ext-direct=dot', '-o', output_path, path]
        
        try:
            subprocess.run(cmd, check=True)
            print(f"Dependency extraction completed for {path}")
            
            # Parse the dot file to create a NetworkX graph
            G = nx.drawing.nx_pydot.read_dot(output_path)
            return G
        except Exception as e:
            print(f"Error extracting dependencies for {path}: {e}")
            # Fallback: use pyreverse
            try:
                output_prefix = os.path.join(OUTPUT_DIR, 'api_dependencies')
                cmd = ['pyreverse', '-o', 'dot', '-p', 'zeeguu', path]
                subprocess.run(cmd, check=True)
                # pyreverse creates classes.dot and packages.dot
                G = nx.drawing.nx_pydot.read_dot('classes.dot')
                os.rename('classes.dot', os.path.join(OUTPUT_DIR, 'classes.dot'))
                os.rename('packages.dot', os.path.join(OUTPUT_DIR, 'packages.dot'))
                return G
            except Exception as e2:
                print(f"Error using pyreverse: {e2}")
                return nx.DiGraph()
    else:
        # For the frontend, use a custom parser for import statements
        G = nx.DiGraph()
        
        import re
        import_patterns = [
            (r"import\s+([\w\s,{}]+)\s+from\s+['\"](.*)['\"]", lambda m: (m.group(2), m.group(1))),
            (r"import\s+['\"](.*)['\"]", lambda m: (m.group(1), None))
        ]
        
        for root, _, files in os.walk(path):
            for file in files:
                if file.endswith(('.js', '.jsx', '.ts', '.tsx')):
                    file_path = os.path.join(root, file)
                    rel_path = os.path.relpath(file_path, path)
                    
                    try:
                        with open(file_path, 'r', encoding='utf-8') as f:
                            content = f.read()
                            
                            for pattern, extractor in import_patterns:
                                for match in re.finditer(pattern, content):
                                    imported_module, _ = extractor(match)
                                    
                                    # Skip external dependencies (non-relative imports)
                                    if not imported_module.startswith('.'):
                                        continue
                                    
                                    # Normalize relative path
                                    target_path = os.path.normpath(os.path.join(os.path.dirname(rel_path), imported_module))
                                    G.add_edge(rel_path, target_path)
                    except Exception as e:
                        print(f"Error parsing {file_path}: {e}")
        
        return G

# Extract backend dependencies
try:
    backend_deps = extract_dependencies(API_PATH, is_backend=True)
    print(f"Backend dependency graph has {backend_deps.number_of_nodes()} nodes and {backend_deps.number_of_edges()} edges")
except Exception as e:
    print(f"Could not extract backend dependencies: {e}")
    backend_deps = nx.DiGraph()

# Extract frontend dependencies
try:
    frontend_deps = extract_dependencies(FRONTEND_PATH, is_backend=False)
    print(f"Frontend dependency graph has {frontend_deps.number_of_nodes()} nodes and {frontend_deps.number_of_edges()} edges")
except Exception as e:
    print(f"Could not extract frontend dependencies: {e}")
    frontend_deps = nx.DiGraph()

## 6. Visualizing Dependency Graphs

Below is a simple visualization of the dependency graphs for the backend and frontend.

In [ ]:
def draw_dependency_graph(G, title="Dependency Graph", figsize=(10, 8)):
    """
    Draws a directed dependency graph using matplotlib.
    """
    plt.figure(figsize=figsize)
    pos = nx.spring_layout(G, k=0.5, iterations=50)
    nx.draw(G, pos, with_labels=True, node_size=500, arrowstyle='->', arrowsize=10)
    plt.title(title)
    plt.show()

# Visualize backend dependencies if available
if backend_deps.number_of_nodes() > 0:
    draw_dependency_graph(backend_deps, title="Backend Dependency Graph")
else:
    print("No backend dependency graph to display.")

# Visualize frontend dependencies if available
if frontend_deps.number_of_nodes() > 0:
    draw_dependency_graph(frontend_deps, title="Frontend Dependency Graph")
else:
    print("No frontend dependency graph to display.")